In [ ]:
import pandas as pd

# Load the provided log file
file_path = './logs(2).csv'

# Attempting to read the file to inspect its contents
try:
    logs_df = pd.read_csv(file_path)
    # Display the first few rows to understand the structure
    logs_df.head(), logs_df.info()
except Exception as e:
    str(e)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17111 entries, 0 to 17110
Data columns (total 10 columns):
 #   Column                                                                                                           Non-Null Count  Dtype 
---  ------                                                                                                           --------------  ----- 
 0   06/Jan/2025                                                                                                      17111 non-null  object
 1   06:02:58                                                                                                         17111 non-null  object
 2   82.81.156.133                                                                                                    17111 non-null  object
 3   GET                                                                                                              17111 non-null  object
 4   /                                       

In [2]:
# Combine date and time into a single timestamp column
logs_df['timestamp'] = pd.to_datetime(logs_df.iloc[:, 0] + ' ' + logs_df.iloc[:, 1], errors='coerce')

# Renaming columns for clarity
logs_df.columns = [
    'date', 'time', 'ip_address', 'http_method', 'url', 'protocol',
    'status_code', 'extra_info_1', 'user_agent', 'extra_info_2', 'timestamp'
]

# Dropping original date and time columns as they're now combined
logs_df = logs_df.drop(columns=['date', 'time'])

# Checking for missing or incomplete data
missing_data_summary = logs_df.isnull().sum()

logs_df.head(), missing_data_summary


(      ip_address http_method         url  protocol  status_code  \
 0  82.81.156.133         GET  /logowanie  HTTP/1.1          200   
 1  82.81.156.133        POST  /logowanie  HTTP/1.1          200   
 2  82.81.156.133         GET  /dashboard  HTTP/1.1          200   
 3   81.13.183.39         GET           /  HTTP/1.1          200   
 4   81.13.183.39         GET  /logowanie  HTTP/1.1          200   
 
                     extra_info_1  \
 0           https://kingbank.pl/   
 1  https://kingbank.pl/logowanie   
 2  https://kingbank.pl/logowanie   
 3                            NaN   
 4           https://kingbank.pl/   
 
                                           user_agent  \
 0  Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...   
 1  Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...   
 2  Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...   
 3  Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...   
 4  Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...   
 
                      

In [3]:
# Handling missing data
# Replace missing `extra_info_1` and `extra_info_2` with 'unknown' as they may indicate useful absence patterns
logs_df['extra_info_1'] = logs_df['extra_info_1'].fillna('unknown')
logs_df['extra_info_2'] = logs_df['extra_info_2'].fillna('unknown')

# Feature engineering: extract key patterns
# Extract domain from `extra_info_1` URLs for potential clustering
logs_df['domain'] = logs_df['extra_info_1'].apply(lambda x: x.split('/')[2] if '//' in x else 'unknown')

# Derive request timing features
logs_df['hour'] = logs_df['timestamp'].dt.hour
logs_df['minute'] = logs_df['timestamp'].dt.minute
logs_df['second'] = logs_df['timestamp'].dt.second

# Encode status codes as categories
logs_df['status_code_cat'] = logs_df['status_code'].astype('category').cat.codes

# Check the processed data
logs_df.head()


,ip_address,http_method,url,protocol,status_code,extra_info_1,user_agent,extra_info_2,timestamp,domain,hour,minute,second,status_code_cat
0,82.81.156.133,GET,/logowanie,HTTP/1.1,200,https://kingbank.pl/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,unknown,2025-01-06 06:03:31,kingbank.pl,6,3,31,0
1,82.81.156.133,POST,/logowanie,HTTP/1.1,200,https://kingbank.pl/logowanie,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,login=tombie3704&password=&action=login,2025-01-06 06:03:36,kingbank.pl,6,3,36,0
2,82.81.156.133,GET,/dashboard,HTTP/1.1,200,https://kingbank.pl/logowanie,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,unknown,2025-01-06 06:03:36,kingbank.pl,6,3,36,0
3,81.13.183.39,GET,/,HTTP/1.1,200,unknown,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,unknown,2025-01-06 06:06:23,unknown,6,6,23,0
4,81.13.183.39,GET,/logowanie,HTTP/1.1,200,https://kingbank.pl/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,unknown,2025-01-06 06:06:55,kingbank.pl,6,6,55,0


In [4]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

# Selecting features for anomaly detection
feature_columns = ['status_code_cat', 'hour', 'minute', 'second']
categorical_columns = ['http_method', 'protocol', 'domain']
encoded_features = []

# Encoding categorical features
encoder = LabelEncoder()
for col in categorical_columns:
    logs_df[col + '_encoded'] = encoder.fit_transform(logs_df[col])
    encoded_features.append(col + '_encoded')

# Final feature set for modeling
model_features = feature_columns + encoded_features

logs_df[model_features]


,status_code_cat,hour,minute,second,http_method_encoded,protocol_encoded,domain_encoded
0,0,6,3,31,0,0,0
1,0,6,3,36,1,0,0
2,0,6,3,36,0,0,0
3,0,6,6,23,0,0,1
4,0,6,6,55,0,0,0
...,...,...,...,...,...,...,...
17106,0,11,32,47,0,0,0
17107,1,11,32,54,1,0,0
17108,0,11,32,54,0,0,1
17109,0,11,32,58,1,0,0


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
import re

# Define custom patterns for SQL injection detection
sql_keywords = ['select', 'union', 'insert', 'update', 'delete', '--', ';--', 'or 1=1', 'drop']

def extract_sql_features(text):
    """
    Identify the presence of SQL-related patterns in the text.
    """
    text = text.lower()  # Convert to lowercase for uniformity
    return sum(1 for keyword in sql_keywords if keyword in text)

# Apply custom SQL feature extraction
logs_df['sql_pattern_count'] = logs_df['url'].apply(extract_sql_features) + \
                               logs_df['extra_info_2'].apply(extract_sql_features)


logs_df


,ip_address,http_method,url,protocol,status_code,extra_info_1,user_agent,extra_info_2,timestamp,domain,hour,minute,second,status_code_cat,http_method_encoded,protocol_encoded,domain_encoded,sql_pattern_count
0,82.81.156.133,GET,/logowanie,HTTP/1.1,200,https://kingbank.pl/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,unknown,2025-01-06 06:03:31,kingbank.pl,6,3,31,0,0,0,0,0
1,82.81.156.133,POST,/logowanie,HTTP/1.1,200,https://kingbank.pl/logowanie,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,login=tombie3704&password=&action=login,2025-01-06 06:03:36,kingbank.pl,6,3,36,0,1,0,0,0
2,82.81.156.133,GET,/dashboard,HTTP/1.1,200,https://kingbank.pl/logowanie,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,unknown,2025-01-06 06:03:36,kingbank.pl,6,3,36,0,0,0,0,0
3,81.13.183.39,GET,/,HTTP/1.1,200,unknown,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,unknown,2025-01-06 06:06:23,unknown,6,6,23,0,0,0,1,0
4,81.13.183.39,GET,/logowanie,HTTP/1.1,200,https://kingbank.pl/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,unknown,2025-01-06 06:06:55,kingbank.pl,6,6,55,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17106,185.74.7.224,GET,/logowanie,HTTP/1.1,200,https://kingbank.pl/,Mozilla/5.0 (iPad; CPU OS 16_5 like Mac OS X) ...,unknown,2025-01-18 11:32:47,kingbank.pl,11,32,47,0,0,0,0,0
17107,81.134.181.48,POST,/wylogowanie,HTTP/1.1,302,https://kingbank.pl/dashboard,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:1...,action=logout,2025-01-18 11:32:54,kingbank.pl,11,32,54,1,1,0,0,0
17108,81.134.181.48,GET,/,HTTP/1.1,200,unknown,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:1...,unknown,2025-01-18 11:32:54,unknown,11,32,54,0,0,0,1,0
17109,185.74.7.224,POST,/logowanie,HTTP/1.1,200,https://kingbank.pl/logowanie,Mozilla/5.0 (iPad; CPU OS 16_5 like Mac OS X) ...,login=karkoz4136&password=&action=login,2025-01-18 11:32:58,kingbank.pl,11,32,58,0,1,0,0,0


In [54]:
from gensim.models import Word2Vec
from gensim.models import FastText
from sklearn.preprocessing import StandardScaler
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.cluster import DBSCAN

# Step 1: Preprocess and Tokenize Text
text_columns = ['url', 'user_agent', 'extra_info_2']
logs_df['combined_text'] = logs_df[text_columns].apply(lambda x: ' '.join(x), axis=1)

# Tokenize text into lists of words
logs_df['tokenized_text'] = logs_df['combined_text'].apply(lambda x: x.lower().split())

# Load Sentence-BERT model
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate sentence embeddings for each log entry
logs_df['sbert_features'] = logs_df['combined_text'].apply(
    lambda text: sbert_model.encode(text)
)

# Step 4: Prepare Features for the Model
# Convert Word2Vec features into a DataFrame
word2vec_features = np.vstack(logs_df['sbert_features'].values)
word2vec_features_df = pd.DataFrame(word2vec_features, columns=[f'sbert_{i}' for i in range(384)])

# Combine Word2Vec features with other features
numerical_features = ['sql_pattern_count']
categorical_encoded_features = ['http_method_encoded', 'protocol_encoded', 'domain_encoded']

all_features_sql = pd.concat([logs_df[numerical_features].reset_index(drop=True),
                          word2vec_features_df], axis=1)
all_features_nosql = word2vec_features_df

print(all_features_sql)


       sql_pattern_count   sbert_0   sbert_1   sbert_2   sbert_3   sbert_4  \
0                      0 -0.028925  0.028378 -0.029137 -0.072334  0.068089   
1                      0 -0.028344  0.003696 -0.029378 -0.092969  0.047750   
2                      0 -0.051145  0.006044 -0.058046 -0.054975  0.061983   
3                      0 -0.044318 -0.001887 -0.027880 -0.033906  0.056138   
4                      0 -0.028925  0.028378 -0.029137 -0.072334  0.068089   
...                  ...       ...       ...       ...       ...       ...   
17106                  0 -0.025647  0.056584 -0.007596 -0.067746  0.100253   
17107                  0  0.020696  0.070344 -0.086134  0.015622  0.052021   
17108                  0 -0.024613  0.010309 -0.059414  0.012388  0.035997   
17109                  0 -0.032696  0.035122 -0.016356 -0.076905  0.089903   
17110                  0 -0.048530  0.042547 -0.029746 -0.055309  0.102429   

        sbert_5   sbert_6   sbert_7   sbert_8  ...  sbert_374  

In [69]:

dbscan = DBSCAN(eps=0.5, min_samples=20)
dbscan_labels = dbscan.fit_predict(all_features_sql)

# Predict anomalies (-1 = anomaly, 1 = normal)
logs_df['anomaly'] = dbscan_labels
logs_df['anomaly'] = logs_df['anomaly'].map({1: 0, -1: 1})  # Map 1 to normal, -1 to anomaly

# Output number of anomalies detected
num_anomalies = logs_df['anomaly'].sum()
print(f"Number of anomalies detected: {num_anomalies}")
anomalous_logs = logs_df[logs_df['anomaly'] == 1]

anomalous_logs.to_excel('anomalous_logs_sql.xlsx')

Number of anomalies detected: 48.0
